# Basic GNN Model Implementation

### Ver 1

In [64]:
import torch
import torch.nn.functional as F
from torch.nn import Parameter
from torch import nn

# degree 함수 구현
def degree(edge_index, num_nodes):
    out_degree = torch.zeros([num_nodes], dtype=torch.float, device=edge_index.device)
    row = edge_index[0]
    out_degree.scatter_add_(0, row, torch.ones_like(edge_index[0].float()))
    return out_degree

class BasicGNN(nn.Module):
    def __init__(self, input_dims, output_dims):
        super().__init__()
        self.input_dims = input_dims
        self.output_dims = output_dims
        self.self_weight = Parameter(torch.FloatTensor(input_dims, output_dims))
        self.neighbor_weight = Parameter(torch.FloatTensor(input_dims, output_dims))
        self.bias = Parameter(torch.FloatTensor(output_dims))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.self_weight)
        nn.init.xavier_uniform_(self.neighbor_weight)
        nn.init.zeros_(self.bias)

    def forward(self, data):
        input_features, edge_index = data.x, data.edge_index
        adjacency = self.normalize(edge_index, input_features.size(0))

        self_support = torch.mm(input_features, self.self_weight)
        neighbor_support = torch.spmm(adjacency, torch.mm(input_features, self.neighbor_weight))
        output = self_support + neighbor_support + self.bias
        # output = torch.relu(output)
        return output

    def normalize(self, edge_index, num_nodes):
        row, col = edge_index
        deg = degree(row, num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        adj = torch.sparse.FloatTensor(edge_index, norm, torch.Size([num_nodes, num_nodes]))
        return adj.to_dense()


### Load Dataset

In [44]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Citeseer', name='Citeseer')

In [ ]:
model = BasicGNN(in_features, out_features)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_func = NLLLoss()

for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    output = model(adjacency, input_features)
    
    # Compute the loss
    loss = loss_func(output[train_mask], labels[train_mask])
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()

In [2]:
def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

def test(model, data):
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    correct = pred[data.test_mask] == data.y[data.test_mask]
    return int(correct.sum()) / int(data.test_mask.sum())

In [8]:
dataset.num_node_features

3703

### Cora DATASET 확인

In [31]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [45]:
data = dataset[0]
data.x

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [46]:
data.edge_index

tensor([[   0,    1,    1,  ..., 3324, 3325, 3326],
        [ 628,  158,  486,  ..., 2820, 1643,   33]])

In [28]:
data.x

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [47]:
data.x.shape[1]

3703

In [66]:
data.x.size(1)

3703

In [51]:
dataset.num_node_features, dataset.num_classes

(3703, 6)

In [48]:
data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])

In [49]:
data.edge_index.shape

torch.Size([2, 9104])

In [52]:
neighbor_weight = Parameter(torch.FloatTensor(3703, 6))
torch.mm(data.x, neighbor_weight).shape

torch.Size([3327, 6])

### Ver 2

In [87]:
import torch
import torch.nn.functional as F
from torch.nn import Parameter
from torch import nn
import torch_scatter

class BasicGNN(nn.Module):
    def __init__(self, input_dims, output_dims):
        super().__init__()
        self.input_dims = input_dims
        self.output_dims = output_dims
        self.self_weight = Parameter(torch.FloatTensor(input_dims, output_dims))
        self.neighbor_weight = Parameter(torch.FloatTensor(input_dims, output_dims))
        self.bias = Parameter(torch.FloatTensor(output_dims))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.self_weight)
        nn.init.xavier_uniform_(self.neighbor_weight)
        nn.init.zeros_(self.bias)

    def forward(self, x, edge_index):
        adj = self.symmetric_normalization(edge_index, x.size(0))

        self_support = torch.mm(x, self.self_weight)
        neighbor_support = torch.spmm(adj, torch.mm(x, self.neighbor_weight))
        output = self_support + neighbor_support + self.bias
        # output = torch.relu(output)
        return output

    def symmetric_normalization(self, edge_index, num_nodes):
        row, col = edge_index

        # Make D
        deg = torch_scatter.scatter_add(torch.ones_like(row), row, dim_size=num_nodes).pow(-0.5)

        # D^(-0.5)AD^(-0.5)
        deg_row = deg[row]
        deg_col = deg[col]
        norm = deg_row * deg_col
        norm_adj = torch.sparse.FloatTensor(edge_index, norm, torch.Size([num_nodes, num_nodes]))

        return norm_adj


In [95]:
D = torch.FloatTensor([[1, 0, 0],
    [0, 0.707, 0],
    [0, 0, 1]])
D.pow(-0.5)
# compute sqrt of 0.707


tensor([[1.0000,    inf,    inf],
        [   inf, 1.1893,    inf],
        [   inf,    inf, 1.0000]])

In [82]:
edge_index = data.edge_index
num_nodes = data.x.size(0)
# torch.sparse.FloatTensor(row=edge_index[0], col=edge_index[1], sparse_sizes=(num_nodes, num_nodes))

row, col = edge_index
deg = torch_scatter.scatter_add(torch.ones_like(row), row, dim_size=num_nodes).pow(-0.5)

# Symmetric normalization: D^(-0.5)AD^(-0.5)
deg_row = deg[row]
deg_col = deg[col]
norm = deg_row * deg_col
norm_adj = torch.sparse.FloatTensor(edge_index, norm, torch.Size([num_nodes, num_nodes]))
norm_adj

tensor(indices=tensor([[   0,    1,    1,  ..., 3324, 3325, 3326],
                       [ 628,  158,  486,  ..., 2820, 1643,   33]]),
       values=tensor([1.0000, 0.2582, 0.1414,  ..., 0.2887, 1.0000, 0.4472]),
       size=(3327, 3327), nnz=9104, layout=torch.sparse_coo)

In [68]:
class Net(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super().__init__()

        self.conv1 = BasicGNN(num_node_features, 16)
        self.conv2 = BasicGNN(16, num_classes)

        # Dropout layer
        self.dropout = torch.nn.Dropout(p=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.dropout(x)
        x = self.conv1(x, edge_index)
        x = torch.nn.functional.relu(x)

        x = self.dropout(x)
        x = self.conv2(x, edge_index)

        return torch.nn.functional.softmax(x, dim=1)

In [65]:
data = dataset[0]
model = BasicGNN(dataset.num_node_features, dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(100):
    train(model, data, optimizer)
    if epoch % 10 == 0:
        acc = test(model, data)
        print('Epoch: {:03d}, Accuracy: {:.4f}'.format(epoch, acc))
print('Epoch: {:03d}, Accuracy: {:.4f}'.format(epoch, acc))

Epoch: 000, Accuracy: 0.0770
Epoch: 010, Accuracy: 0.0770
Epoch: 020, Accuracy: 0.0770
Epoch: 030, Accuracy: 0.0770
Epoch: 040, Accuracy: 0.0770
Epoch: 050, Accuracy: 0.0770
Epoch: 060, Accuracy: 0.0770
Epoch: 070, Accuracy: 0.0770
Epoch: 080, Accuracy: 0.0770
Epoch: 090, Accuracy: 0.0770
Epoch: 099, Accuracy: 0.0770


In [85]:
data = dataset[0]
model = Net(dataset.num_node_features, dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(100):
    train(model, data, optimizer)
    if epoch % 10 == 0:
        acc = test(model, data)
        print('Epoch: {:03d}, Accuracy: {:.4f}'.format(epoch, acc))
print('Epoch: {:03d}, Accuracy: {:.4f}'.format(epoch, acc))

Epoch: 000, Accuracy: 0.2940
Epoch: 010, Accuracy: 0.6420
Epoch: 020, Accuracy: 0.6420
Epoch: 030, Accuracy: 0.6670
Epoch: 040, Accuracy: 0.6460
Epoch: 050, Accuracy: 0.6620
Epoch: 060, Accuracy: 0.6800
Epoch: 070, Accuracy: 0.6760
Epoch: 080, Accuracy: 0.6520
Epoch: 090, Accuracy: 0.6740
Epoch: 099, Accuracy: 0.6740
